# animating dimension reduction of spike times as point processes
Tim Tyree<br>
2.20.2021

In [23]:
#automate the boring stuff
from lib.getterdone import *

In [24]:
# from lib import *

In [42]:
#search_for_filename
# input_file_name=search_for_file()
#manual data has an ugly dtype
# input_file_name='/Users/timothytyree/Documents/GitHub/neurophysics/notebooks/Data/SRT_CollabData/SortedData_Manual/catTempPopStorage_full.mat'
#early autogen data
input_file_name='/Users/timothytyree/Documents/GitHub/neurophysics/notebooks/Data/SRT_CollabData/SortedData_AutoGen/Archie_SRT_Set226_subset1_201020_142509_UnitStorage.mat'

In [45]:
# import scipy.io as sio
# mat_contents = sio.loadmat(input_file_name)
# sorted(mat_contents.keys())

In [82]:
# #TODO: import mike's .mat file
import h5py
field_lst=['chanNum', 'featSpace', 'spikeInds', 'spikeTimes', 'trialFR', 'trialPSTH', 'trialSpks', 'wvfrms']
with h5py.File(input_file_name, 'r') as f:
#     print(f.keys())
#     print(mat:=f['SRTunitStorage'].keys())#.fletcher32)#.items())#__dir__())
    print(obj:=f['SRTunitStorage']['spikeTimes'])
    #     mat=f['categoricalPopStorage']
    #     print(mat.__dir__())#.keys())
#     print(mat.popitem['chunks'])
    str1 = ''.join(chr(i) for i in obj)
    print( str1 )


<HDF5 dataset "spikeTimes": shape (195, 1), type "|O">


TypeError: only integer scalar arrays can be converted to a scalar index

In [83]:
mat.

SyntaxError: invalid syntax (<ipython-input-83-bcdfa369c6ec>, line 1)

In [84]:
# #represent spiketime trial as a point process in terms of an l x m x n numpy array


In [85]:
#TODO: naïve eigenvalue decomposition of ^that

import mat73
data_dict = mat73.loadmat(input_file_name)

In [20]:
# !pip install mat73
# !pip install hdf5storage


  Using cached hdf5storage-0.1.15-py2.py3-none-any.whl (56 kB)


In [22]:
#TODO: animate a umap with the number of neighbors \propto time
import hdf5storage
mat = hdf5storage.loadmat(input_file_name)
